In [2]:
import pandas as pd

In [3]:
df=pd.read_csv('/content/drive/MyDrive/Major project/archive(1)/Tweets.csv')

In [4]:
df.drop(columns=['tweet_id','airline_sentiment_confidence','negativereason','negativereason_confidence','airline','airline_sentiment_gold','name','negativereason_gold','retweet_count','tweet_coord','tweet_created','tweet_location','user_timezone'],inplace = True)

In [5]:
df_filtered=df[df['airline_sentiment'] != 'neutral']
df_filtered.head(20)

,airline_sentiment,text
1,positive,@VirginAmerica plus you've added commercials t...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
5,negative,@VirginAmerica seriously would pay $30 a fligh...
6,positive,"@VirginAmerica yes, nearly every time I fly VX..."
8,positive,"@virginamerica Well, I didn't…but NOW I DO! :-D"
9,positive,"@VirginAmerica it was amazing, and arrived an ..."
11,positive,@VirginAmerica I &lt;3 pretty graphics. so muc...
12,positive,@VirginAmerica This is such a great deal! Alre...
13,positive,@VirginAmerica @virginmedia I'm flying your #f...


In [6]:
import nltk
nltk.download('stopwords')
stopword_list=nltk.corpus.stopwords.words('english')
stopword_list.remove("no")
stopword_list.remove("not")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
!pip install contractions

     |████████████████████████████████| 327kB 30.0MB/s 
     |████████████████████████████████| 266kB 37.2MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85398 sha256=875b6b5db8ac922a93dbf0dca89c950d97e2ee7c2823a1dc76e2fc644bf52b5b
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [8]:
import requests
from bs4 import BeautifulSoup

In [9]:
def html_tag(text):
  soup=BeautifulSoup(text,"html.parser")
  new_text=soup.get_text()
  return new_text

In [10]:
import contractions
def con(text):
  expand=contractions.fix(text)
  return expand

In [11]:
import re
def remove_sp(text):
  pattern=r'[^A-Za-z0-9\s]'
  text=re.sub(pattern,'',text)
  return text

In [12]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()

In [13]:
def remove_stopwords(text):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip()for token in tokens]
  filtered_tokens=[token for token in tokens if token not in stopword_list]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text

In [14]:
df_filtered.airline_sentiment=df_filtered.airline_sentiment.apply(lambda x:x.lower())
df_filtered.airline_sentiment=df_filtered.airline_sentiment.apply(html_tag)
df_filtered.airline_sentiment=df_filtered.airline_sentiment.apply(con)
df_filtered.airline_sentiment=df_filtered.airline_sentiment.apply(remove_sp)
df_filtered.airline_sentiment=df_filtered.airline_sentiment.apply(remove_stopwords)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [15]:
df_filtered['airline_sentiment'].value_counts()

negative    9178
positive    2363
Name: airline_sentiment, dtype: int64

In [16]:
x = df_filtered.iloc[:,1]
y = df_filtered.iloc[:,0]

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0,stratify=y)

In [18]:
print(x_train.shape)
print(x_test.shape)

(8078,)
(3463,)


In [19]:
import numpy as np
np.unique(y_train,return_counts=True)

(array(['negative', 'positive'], dtype=object), array([6424, 1654]))

In [20]:
np.unique(y_test,return_counts=True)

(array(['negative', 'positive'], dtype=object), array([2754,  709]))

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
sa_model = Pipeline([('tfidf',TfidfVectorizer()),('model',SVC())])

In [22]:
sa_model.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('model',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', d

In [23]:
y_pred = sa_model.predict(x_test)
y_pred

array(['negative', 'negative', 'negative', ..., 'negative', 'negative',
       'negative'], dtype=object)

In [24]:
y_test

3512     negative
10159    negative
9752     negative
2876     negative
14311    negative
           ...   
4709     positive
10002    negative
6425     negative
1646     positive
2576     positive
Name: airline_sentiment, Length: 3463, dtype: object

In [25]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [26]:
accuracy_score(y_pred,y_test)*100

91.48137453075368

In [27]:
confusion_matrix(y_pred,y_test)

array([[2725,  266],
       [  29,  443]])

In [28]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

    negative       0.99      0.91      0.95      2991
    positive       0.62      0.94      0.75       472

    accuracy                           0.91      3463
   macro avg       0.81      0.92      0.85      3463
weighted avg       0.94      0.91      0.92      3463



In [29]:
sentiment_m = df[df['airline_sentiment']=='negative']
sentiment_m.iloc[6].values

array(['negative',
       '@VirginAmerica you guys messed up my seating.. I reserved seating with my friends and you guys gave my seat away ... 😡 I want free internet'],
      dtype=object)

In [30]:
sa_model.predict(['@VirginAmerica you guys messed up my seating.. I reserved seating with my friends and you guys gave my seat away ... 😡 I want free internet'])

array(['negative'], dtype=object)

In [31]:
!pip install pipreqs

In [32]:
!pipreqs /content/drive/MyDrive/Major project

Usage:
    pipreqs [options] [<path>]


In [33]:
cd /content/drive/MyDrive/Major project

/content/drive/MyDrive/Major project


In [34]:
import joblib
joblib.dump(sa_model,'Tweets')

['Tweets']

In [35]:
!pip install streamlit --quiet
!pip install pyngrok==4.1.1 --quiet
from pyngrok import ngrok

     |████████████████████████████████| 8.2MB 26.8MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 163kB 46.1MB/s 
     |████████████████████████████████| 112kB 44.2MB/s 
     |████████████████████████████████| 4.2MB 35.6MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 122kB 43.8MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.3 which is incompatible.


In [36]:
%%writefile Sentiments.py
import streamlit as st
import sklearn
import joblib
model = joblib.load('Tweets')
st.title('Tweets Classifier')
ip = st.text_input('Enter your message')
op = model.predict([ip])
if st.button('Predict'):
  st.title(op[0])

Overwriting Sentiments.py


In [37]:
!nohup streamlit run Sentiments.py &
url=ngrok.connect(port='8501')
url

nohup: appending output to 'nohup.out'


'http://334ad410a4d9.ngrok.io'